<a href="https://colab.research.google.com/github/park-geun-hyeong/Computer_Vision/blob/main/CH2/FasterRCNN_opencv_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Git Clone


In [1]:
!git clone https://github.com/chulminkw/DLCV.git

Cloning into 'DLCV'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 234 (delta 32), reused 0 (delta 0), pack-reused 159
Receiving objects: 100% (234/234), 142.72 MiB | 35.00 MiB/s, done.
Resolving deltas: 100% (89/89), done.


##Library Import

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import os
import cv2
from PIL import Image
from tqdm import tqdm

In [3]:
path = '/content/DLCV/data/' #img directory 설정
video_path = path+'video/John_Wick_small.mp4'

In [4]:
os.mkdir('/content/DLCV/faster_RCNN') # make new directory
os.chdir('/content/DLCV/faster_RCNN') # change home directory for data download 
os.getcwd()

'/content/DLCV/faster_RCNN'

## Pre_trained Model download(Resnet 50)

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/faster_rcnn_resnet50_coco_2018_01_28.tar.gz
!wget https://raw.githubusercontent.com/opencv/opencv_extra/master/testdata/dnn/faster_rcnn_resnet50_coco_2018_01_28.pbtxt

In [ ]:
!echo "##### uncompressing tar file"
!tar -xvf faster_rcnn*.gz ##tgr file zip out

In [7]:
!mv /content/DLCV/faster_RCNN/faster_rcnn*.pbtxt /content/DLCV/faster_RCNN/faster_rcnn_resnet50_coco_2018_01_28/graph.pbtxt #move file

In [8]:
!rm *.gz #tgr file remove

In [9]:
sorted(os.listdir('/content/DLCV/faster_RCNN/faster_rcnn_resnet50_coco_2018_01_28')) #directiory search

['checkpoint',
 'frozen_inference_graph.pb',
 'graph.pbtxt',
 'model.ckpt.data-00000-of-00001',
 'model.ckpt.index',
 'model.ckpt.meta',
 'pipeline.config',
 'saved_model']

## Tensorflow inference model loading

In [10]:
model_path = '/content/DLCV/faster_RCNN/faster_rcnn_resnet50_coco_2018_01_28/'

cv_net = cv2.dnn.readNetFromTensorflow(model_path+'frozen_inference_graph.pb',model_path+'graph.pbtxt')

## Lablel Mapping

In [11]:
labels_to_names_0 = {0:'person',1:'bicycle',2:'car',3:'motorcycle',4:'airplane',5:'bus',6:'train',7:'truck',8:'boat',9:'traffic light',
                    10:'fire hydrant',11:'street sign',12:'stop sign',13:'parking meter',14:'bench',15:'bird',16:'cat',17:'dog',18:'horse',19:'sheep',
                    20:'cow',21:'elephant',22:'bear',23:'zebra',24:'giraffe',25:'hat',26:'backpack',27:'umbrella',28:'shoe',29:'eye glasses',
                    30:'handbag',31:'tie',32:'suitcase',33:'frisbee',34:'skis',35:'snowboard',36:'sports ball',37:'kite',38:'baseball bat',39:'baseball glove',
                    40:'skateboard',41:'surfboard',42:'tennis racket',43:'bottle',44:'plate',45:'wine glass',46:'cup',47:'fork',48:'knife',49:'spoon',
                    50:'bowl',51:'banana',52:'apple',53:'sandwich',54:'orange',55:'broccoli',56:'carrot',57:'hot dog',58:'pizza',59:'donut',
                    60:'cake',61:'chair',62:'couch',63:'potted plant',64:'bed',65:'mirror',66:'dining table',67:'window',68:'desk',69:'toilet',
                    70:'door',71:'tv',72:'laptop',73:'mouse',74:'remote',75:'keyboard',76:'cell phone',77:'microwave',78:'oven',79:'toaster',
                    80:'sink',81:'refrigerator',82:'blender',83:'book',84:'clock',85:'vase',86:'scissors',87:'teddy bear',88:'hair drier',89:'toothbrush',
                    90:'hair brush'}

##Confirm Video

In [15]:
video_output_path = path+'video/new_John_wick.avi'

cap = cv2.VideoCapture(video_path) #VideoCapture 함수를 사용하여 video 객체로 저장

codec = cv2.VideoWriter_fourcc(*'XVID') # 비디오 인코딩을 XVID형태로 만들기

video_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
video_FPS = cap.get(cv2.CAP_PROP_FPS)

video_writer = cv2.VideoWriter(video_output_path, codec, video_FPS, video_size )
print("frame_count: {}".format(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("Size: {}".format(video_size))
print("FPS: {}".format(video_FPS))

frame_count: 58.0
Size: (1072, 460)
FPS: 30.00051725029742


## Process Video Detection per Frame

In [16]:
red = (255,0,0)
green = (0,255,0)
import time

while True: #무한루프

    has_frame,img_frame = cap.read() #videoCapture을 read할경우 두개의 객체가 반환된다.(frame 반환여부, 실제 frame 정보)
    if not has_frame:
        print("NO more Frame to process")
        break

    row = img_frame.shape[0]
    col = img_frame.shape[1]

    cv_net.setInput(cv2.dnn.blobFromImage(img_frame, swapRB=True, crop=False))

    start=time.time()

    cv_out = cv_net.forward()
    frame_index = 0

    for detection in cv_out[0,0,:,:]:
        class_num = int(detection[1])
        score = float(detection[2])

        if score>0.5:
            left = int(detection[3]*col)
            top = int(detection[4]*row)
            right = int(detection[5]*col)
            bottom = int(detection[6]*row)

            caption = "{} ,{:.4f}".format(labels_to_names_0[class_num],score)
            cv2.rectangle(img_frame, (left,top),(right,bottom),color=green, thickness=2)
            cv2.putText(img_frame, caption, (left,top-5),cv2.FONT_HERSHEY_SIMPLEX, 0.4, color=red, thickness=1)

    print("time: {:.4f}".format(time.time()-start))
    video_writer.write(img_frame)

video_writer.release()
cap.release()


time: 8.5742
time: 8.6807
time: 8.5532
time: 8.6053
time: 8.5091
time: 8.5401
time: 8.5875
time: 8.5553
time: 8.5599
time: 8.6511
time: 8.4751
time: 8.5248
time: 8.5953
time: 9.3628
time: 8.9872
time: 8.6335
time: 8.8667
time: 8.7395
time: 9.0296
time: 9.1422
time: 8.5774
time: 8.6261
time: 8.5950
time: 8.6443
time: 8.4959
time: 8.4964
time: 8.5393
time: 8.5541
time: 8.6745
time: 8.5691
time: 8.5510
time: 8.8022
time: 8.7673
time: 8.8555
time: 8.5197
time: 8.6193
time: 8.6143
time: 8.5805
time: 8.5488
time: 8.4698
time: 8.5724
time: 9.7150
time: 8.8043
time: 8.6511
time: 8.6171
time: 8.6574
time: 8.4462
time: 8.6350
time: 8.4951
time: 8.5931
time: 8.6773
time: 8.4441
time: 8.5466
time: 8.5725
time: 8.9373
time: 8.5897
time: 8.5511
time: 8.5815
NO more Frame to process
